## Get a pre-saved statistical t-map and run FDR correction

In [1]:
# Compare and analyze seed-based connectivity maps
# import
from nilearn import datasets
from nilearn import surface
from nilearn import plotting
from nilearn import image
import numpy as np
import pandas as pd
import glob
import os
import nilearn
from nilearn import signal
from scipy import stats
import nibabel as nib
from os.path import exists
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats as st

In [195]:
# Load a single subject map from NSD
# Convert tval to pval
# Run fdr
# Get the threshold tval for the corrected pval
cur_dir = '/scratch/groups/jyeatman/NSD/analysis/clean_18P_scrub025_butter/'
#cur_dir = '/oak/stanford/groups/jyeatman/NSD/analysis/clean_18P_scrub025_butter/'
subs = ['subj01','subj02','subj03','subj04','subj05','subj06','subj07','subj08']
alpha = 0.01

In [196]:
t_subs_lh = np.zeros(len(subs))
t_subs_rh = np.zeros(len(subs))
t_subs_both = np.zeros(len(subs))
t_subs_lh

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [197]:
t_subs_lh = np.zeros(len(subs))
t_subs_rh = np.zeros(len(subs))
t_subs_both = np.zeros(len(subs))

for ind,sub in enumerate(subs):
    cur_file = glob.glob(cur_dir + sub + '_T-test_VWFA1_VWFA2*lh_curv.curv')[0]
    #sub_count = 20
    rid = cur_file.find('runs')
    sub_count = int(cur_file[(rid-2):(rid)])
    print('number of runs per subject: ' + str(sub_count))
    # load map
    tmap_lh = surface.load_surf_data(cur_file)
    t_threshold_lh = get_fdr_thresh(tmap_lh,sub_count,alpha)
    t_subs_lh[ind] = t_threshold_lh
    
    file_rh = str.replace(cur_file,'lh','rh')
    tmap_rh = surface.load_surf_data(file_rh)
    t_threshold_rh = get_fdr_thresh(tmap_rh,sub_count,alpha)
    t_subs_rh[ind] = t_threshold_rh
    
    t_thrshold_both = get_fdr_thresh(np.concatenate((tmap_lh,tmap_rh)),sub_count,alpha)
    t_subs_both[ind] = t_thrshold_both
    
fdr_df = pd.DataFrame({"sub":subs,"LH":t_subs_lh,"RH":t_subs_rh,"BOTH":t_subs_both})

number of runs per subject: 20
New byte order: <
           tval     porig      pcor  pass
78385 -4.978573  0.000083  0.009999  True
[4.97857332]
New byte order: <
            tval     porig      pcor  pass
114506  5.090777  0.000065  0.009999  True
[5.09077692]
            tval    porig      pcor  pass
184712 -5.059829  0.00007  0.009999  True
[5.05982924]
number of runs per subject: 16
New byte order: <
           tval         porig      pcor  pass
24201  9.196889  1.485883e-07  0.009821  True
[9.19688892]
New byte order: <
            tval         porig      pcor  pass
29021 -12.974217  1.476977e-09  0.004582  True
[12.97421742]
             tval         porig     pcor  pass
268654 -12.974217  1.476977e-09  0.00966  True
[12.97421742]
number of runs per subject: 17
New byte order: <
            tval     porig      pcor  pass
81258  -5.606836  0.000039  0.009993  True
178515 -5.606795  0.000039  0.009993  True
[5.60683584 5.60679483]
New byte order: <
            tval     porig  pcor

In [198]:
fdr_df

,sub,LH,RH,BOTH
0,subj01,4.978573,5.090777,5.059829
1,subj02,9.196889,12.974217,12.974217
2,subj03,5.606836,5.892994,5.764382
3,subj04,4.790569,4.838929,4.841599
4,subj05,18.958195,NaN,18.958195
5,subj06,5.538608,5.784452,5.681263
6,subj07,4.959482,5.097779,5.053355
7,subj08,5.148047,5.373221,5.275608


In [161]:
def get_fdr_thresh(tmap,sub_count,alpha):
    ### TBD write docstring
    # this function gets a statistical map (a brain image on the surface, a .curv file)
    # and calulates the FDR threshold for significance for a given alpha and the N on which the map was created
    ###
    
    # I got a weird error about the Endian-ness of the tmap, so if it's big endian we'll swap the byte order
    if tmap.dtype.byteorder == '>':
        tmap=tmap.newbyteorder().byteswap()
        print('New byte order: ' + tmap.dtype.byteorder)

    # convert t values to two-tail p-values
    pmap = stats.t.sf(abs(tmap), sub_count-1)*2
    pmap.shape # shape should be number of vertices

    # method can be 'n' or 'p'/indep - the default is to assume that the comparisons are independent. 
    # In the case of a brain map they are not independent, so I used the other method
    # https://www.statsmodels.org/stable/generated/statsmodels.stats.multitest.fdrcorrection.html#statsmodels.stats.multitest.fdrcorrection
    pcor = st.multitest.fdrcorrection(pmap, alpha=alpha, method='n', is_sorted=False)

    # create dataframe to sort everything together
    df = pd.DataFrame({"tval":tmap,"porig":pmap,"pcor":pcor[1],"pass":pcor[0]})
    df.sort_values(by="tval",axis=0,inplace=True)
    pass_df = df.loc[df['pass']==True,]
    if pass_df.shape[0]<1:
        print('No vertex passed correction at alpha')
        t_threshold = np.NaN
        return t_threshold
    else:
        #threshold = pass_df['tval'][pass_df['pcor']== pass_df['pcor'].max()]
        threshold = pass_df.query('pcor == pcor.max()')
        print(threshold)
        # to double check
        t_threshold = stats.t.isf(threshold.porig/2, sub_count-1)
        print(t_threshold)
        return t_threshold[0]

In [ ]:
# syntax to go back to t-values that correspond to the corrected p values- not sure it makes sense
# tcor = stats.t.isf(df['pcor']/2, sub_count-1)
# tcor = tcor*np.sign(df['tval'])
# df['tcor'] = tcor
# df.head(2000)